In [4]:
pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 KB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

from __future__ import absolute_import
from kafka import KafkaProducer

from time import sleep

In [6]:
int_ip = '10.128.0.30:9092'               
topic = 'lab7topic'                   
file_path = 'gs://me18b183_bdl/lab7kafka/Iris.csv' 

In [7]:
producer = KafkaProducer(bootstrap_servers=[int_ip], value_serializer=lambda x: x.encode('utf-8'))

In [8]:
spark = SparkSession \
    .builder \
    .appName("lab7producer") \
    .getOrCreate()

schema = StructType() \
    .add("SepalLength", FloatType()) \
    .add("SepalWidth", FloatType()) \
    .add("PetalLength", FloatType()) \
    .add("PetalWidth", FloatType()) \
    .add("Species", StringType())

In [9]:
Iris = spark.read.csv(file_path, header=True, schema=schema)

for entry in Iris.toJSON().collect():
    producer.send(topic, value=entry)
    sleep(1)